In [ ]:
import pypsa
import yaml
import cartopy
import sys
import re
import os

import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl

from itertools import product
from matplotlib.lines import Line2D
from matplotlib.patches import FancyArrowPatch
import matplotlib.patches as mpatches
from matplotlib.transforms import Bbox

from vresutils.costdata import annuity

PATH = "../workflows/pypsa-eur-sec/"

sys.path.append(os.path.join(PATH, "scripts/"))
from plot_summary import rename_techs

plt.style.use(["bmh", "matplotlibrc"])
xr.set_options(display_style="html")

%matplotlib inline

In [ ]:
CLUSTERS = 181
LV_OPTS = "Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10"
OUTPUT = "../results/graphics-20221227/"

MAIN_SCENARIOS = PATH + "results/20221227-main"
DEC_SCENARIOS = PATH + "results/20221227-decentral"
LV_SCENARIOS = PATH + "results/20221227-lv"
ONW_SCENARIOS = PATH + "results/20221227-onw"
GAS_SCENARIOS = PATH + "results/20221227-gas"
IMP_SCENARIOS = PATH + "results/20221227-import"
SHP_SCENARIOS = PATH + "results/20221227-shipping"
COST_SCENARIOS = PATH + "results/20221227-costs"
TIME_SCENARIOS = PATH + "results/20221227-time"
SPACE_SCENARIOS = PATH + "results/20221227-spatial"
OLD_SCENARIOS = "../workflows-rev0/pypsa-eur-sec/results/20211218-181-h2"

In [ ]:
latex = {}

In [ ]:
with open(PATH + "config.main.yaml") as file:
    config = yaml.safe_load(file)

In [ ]:
tech_colors = config["plotting"]["tech_colors"]

In [ ]:
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

In [ ]:
def rename_techs_tyndp(tech):
    tech = rename_techs(tech)
    if "heat pump" in tech or "resistive heater" in tech:
        return "power-to-heat"
    elif tech in ["H2 Electrolysis"]:  # , "H2 liquefaction"]:
        return "power-to-hydrogen"
    elif "H2 pipeline" in tech:
        return "H2 pipeline"
    elif tech == "H2":
        return "H2 storage"
    elif tech in ["OCGT", "CHP", "gas boiler", "H2 Fuel Cell"]:
        return "gas-to-power/heat"
    # elif "solar" in tech:
    #    return "solar"
    elif tech in ["Fischer-Tropsch", "methanolisation"]:
        return "power-to-liquid"
    elif "offshore wind" in tech:
        return "offshore wind"
    elif "SMR" in tech:
        return tech.replace("SMR", "steam methane reforming")
    elif "DAC" in tech:
        return "direct air capture"
    elif "CC" in tech or "sequestration" in tech:
        return "carbon capture"
    elif tech == "oil" or tech == "gas":
        return "fossil oil and gas"
    else:
        return tech

In [ ]:
preferred_order = pd.Index(
    [
        "transmission lines",
        "electricity distribution grid",
        "fossil oil and gas",
        "hydroelectricity",
        "hydro reservoir",
        "run of river",
        "pumped hydro storage",
        "solid biomass",
        "biogas",
        "onshore wind",
        "offshore wind",
        "offshore wind (AC)",
        "offshore wind (DC)",
        "solar PV",
        "solar thermal",
        "solar rooftop",
        "solar",
        "building retrofitting",
        "ground heat pump",
        "air heat pump",
        "heat pump",
        "resistive heater",
        "power-to-heat",
        "gas-to-power/heat",
        "CHP",
        "OCGT",
        "gas boiler",
        "gas",
        "natural gas",
        "helmeth",
        "methanation",
        "power-to-gas",
        "power-to-hydrogen",
        "H2 pipeline",
        "H2 liquefaction",
        "H2 storage",
        "hydrogen storage",
        "power-to-liquid",
        "battery storage",
        "hot water storage",
        "CO2 sequestration",
        "CCS",
        "carbon capture and sequestration",
        "DAC",
        "direct air capture",
    ]
)

In [ ]:
def parse_index(c, with_resolution=False):

    clusters = c[0]

    lv = c[1]

    match = re.search(r"onwind\+p([0-9.]*)", c[2])
    onw = 100.0 if match is None else 100 * float(match.groups()[0])

    h2 = "no H2 grid" if "noH2network" in c[2] else "H2 grid"

    to_return = (clusters, lv, onw, h2)

    if with_resolution:
        match = re.findall(r"(\d+)H", c[2])
        to_return += (int(match[0]),)

    return to_return

In [ ]:
def load_decentral():

    costs = pd.read_csv(
        DEC_SCENARIOS + "/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
    )

    costs = costs.xs("2050", level="planning_horizon", axis=1)

    costs.columns = pd.MultiIndex.from_tuples(
        [parse_index(c) for c in costs.columns], names=["clusters", "lv", "onw", "h2"]
    )

    costs = costs.xs(str(CLUSTERS), level="clusters", axis=1)

    df = costs.groupby(level=2).sum().div(1e9)

    df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

    df = df.xs(100, level="onw", axis=1, drop_level=False)

    df.sum() / df.sum().min()

    to_drop = df.index[df.max(axis=1).fillna(0.0) < 1.2]
    print(to_drop)
    df.drop(to_drop, inplace=True)

    order = preferred_order.intersection(df.index).append(
        df.index.difference(preferred_order)
    )
    df = df.loc[order]

    tech_colors = config["plotting"]["tech_colors"]
    colors = [tech_colors[i] for i in df.index]

    df.columns = df.columns.get_level_values(2)
    df.columns.name = ""

    return df

In [ ]:
costs = pd.read_csv(
    DEC_SCENARIOS + "/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
)

costs = costs.xs((str(CLUSTERS), "2050"), level=["cluster", "planning_horizon"], axis=1)

In [ ]:
costs

In [ ]:
def load_main(
    scenarios=None, clusters=None, rename=True, with_resolution=False, with_space=False
):

    if scenarios is None:
        scenarios = MAIN_SCENARIOS

    if clusters is None:
        clusters = CLUSTERS

    horizon = "2030" if "rev0" in scenarios else "2050"

    costs = pd.read_csv(
        scenarios + f"/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
    )

    costs = costs.xs(horizon, level="planning_horizon", axis=1)

    names = ["clusters", "lv", "onw", "h2"]
    if with_resolution:
        names += ("res",)

    costs.columns = pd.MultiIndex.from_tuples(
        [parse_index(c, with_resolution) for c in costs.columns], names=names
    )

    if not with_space:
        costs = costs.xs(str(clusters), level="clusters", axis=1)

    df = costs.groupby(level=2).sum().div(1e9)

    if rename:
        df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

    to_drop = df.index[df.max(axis=1).fillna(0.0) < 1.2]
    print(to_drop)
    df.drop(to_drop, inplace=True)

    order = preferred_order.intersection(df.index).append(
        df.index.difference(preferred_order)
    )
    df = df.loc[order]

    if "-imp" in scenarios:
        # imports for methanol, kerosene and naphtha at 120 €/MWh
        print("add import costs")
        df.loc["green e-fuel imports"] = (1026.64 + 546.36) * 120e6 / 1e9  # bn€/a
        tech_colors["green e-fuel imports"] = "#46caf0"

    return df

In [ ]:
def load_main_capacities(
    scenarios=None, clusters=None, rename=True, with_resolution=False, with_space=False
):

    if scenarios is None:
        scenarios = MAIN_SCENARIOS

    if clusters is None:
        clusters = CLUSTERS

    horizon = "2030" if "rev0" in scenarios else "2050"

    df = pd.read_csv(
        scenarios + f"/csvs/capacities.csv", header=[0, 1, 2, 3], index_col=[0, 1]
    )

    df = df.xs(horizon, level="planning_horizon", axis=1)

    names = ["clusters", "lv", "onw", "h2"]
    if with_resolution:
        names += ("res",)

    df.columns = pd.MultiIndex.from_tuples(
        [parse_index(c, with_resolution) for c in df.columns], names=names
    )

    if not with_space:
        df = df.xs(str(clusters), level="clusters", axis=1)

    if rename:
        grouper = [
            df.index.get_level_values(0),
            df.index.get_level_values(1).map(rename_techs_tyndp),
        ]
        df = df.groupby(grouper).sum()

    to_drop = df.index[df.max(axis=1).fillna(0.0) < 10]
    df.drop(to_drop, inplace=True)

    twh = df.xs("stores", level=0).div(1e6)  # TWh

    to_drop = [
        "CCS",
        "biogas",
        "co2",
        "fossil oil and gas",
        "solid biomass",
    ]
    twh.drop(twh.index.intersection(to_drop), inplace=True)

    gw = df.drop(["stores", "lines"]).groupby(level=1).sum().div(1e3)  # GW

    to_drop = [
        "fossil oil and gas",
        "transmission lines",
        "DAC",
        "direct air capture",
        "H2 pipeline",
        "H2 pipeline retrofitted",
        "CCS",
        "carbon capture" "biogas",
        "gas for industry",
        "hot water storage",
        "solid biomass for industry",
        "process emissions",
    ]
    gw.drop(gw.index.intersection(to_drop), inplace=True)

    return gw, twh

In [ ]:
def load_main_energy(
    scenarios=None, clusters=None, rename=True, with_resolution=False, with_space=False
):

    if scenarios is None:
        scenarios = MAIN_SCENARIOS

    if clusters is None:
        clusters = CLUSTERS

    horizon = "2030" if "rev0" in scenarios else "2050"

    df = pd.read_csv(
        scenarios + f"/csvs/energy.csv", header=[0, 1, 2, 3], index_col=[0, 1]
    )

    df = df.xs(horizon, level="planning_horizon", axis=1)

    names = ["clusters", "lv", "onw", "h2"]
    if with_resolution:
        names += ("res",)

    df.columns = pd.MultiIndex.from_tuples(
        [parse_index(c, with_resolution) for c in df.columns], names=names
    )

    if not with_space:
        df = df.xs(str(clusters), level="clusters", axis=1)

    df = df.groupby(level=1).sum().div(1e6)  # TWh

    if rename:
        df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

    to_drop = df.index[df.abs().max(axis=1).fillna(0.0) < 10]
    df.drop(to_drop, inplace=True)

    order = preferred_order.intersection(df.index).append(
        df.index.difference(preferred_order)
    )
    df = df.loc[order]

    if "-imp" in scenarios:
        # imports for methanol, kerosene and naphtha
        df.loc["green e-fuel imports"] = 1026.64 + 546.36  # TWh
        tech_colors["green e-fuel imports"] = "#46caf0"

    return df

In [ ]:
def load_main_cfs(
    scenarios=None, clusters=None, with_resolution=False, with_space=False
):

    if scenarios is None:
        scenarios = MAIN_SCENARIOS

    if clusters is None:
        clusters = CLUSTERS

    horizon = "2030" if "rev0" in scenarios else "2050"

    df = pd.read_csv(
        scenarios + f"/csvs/cfs.csv", header=[0, 1, 2, 3], index_col=[0, 1]
    )

    df = df.xs(horizon, level="planning_horizon", axis=1)

    names = ["clusters", "lv", "onw", "h2"]
    if with_resolution:
        names += ("res",)

    df.columns = pd.MultiIndex.from_tuples(
        [parse_index(c, with_resolution) for c in df.columns], names=names
    )

    if not with_space:
        df = df.xs(str(clusters), level="clusters", axis=1)

    df = df.groupby(level=1).sum()

    return df

## For CSV Export

In [ ]:
def export_csv(scenario):

    suffix = scenario.split("-")[-1]

    gw, twh = load_main_capacities(rename=False)

    to_keep = [
        "H2",
        "Li ion",
        "battery",
        "home battery",
        "residential rural water tanks",
        "residential urban decentral water tanks",
        "services rural water tanks",
        "services urban decentral water tanks",
        "urban central water tanks",
    ]
    twh.loc[to_keep].xs(100.0, level="onw", axis=1).groupby(
        rename_techs_tyndp
    ).sum().to_csv(f"{OUTPUT}/capacities-twh.csv")

    gw.xs(100.0, level="onw", axis=1).groupby(rename_techs_tyndp).sum().to_csv(
        f"{OUTPUT}/capacities-gw.csv"
    )

    load_main_energy().xs(100.0, level="onw", axis=1).div(1e6).groupby(
        rename_techs_tyndp
    ).sum().to_csv(f"{OUTPUT}/energy.csv")

    load_main().xs(100.0, level="onw", axis=1).to_csv(f"{OUTPUT}/costs.csv")

In [ ]:
for scenario in [
    MAIN_SCENARIOS,
    GAS_SCENARIOS,
    IMP_SCENARIOS,
    SHP_SCENARIOS,
    COST_SCENARIOS,
]:
    export_csv(scenario)

## Line Volume Sensitivity

In [ ]:
costs = pd.read_csv(
    LV_SCENARIOS + "/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
)

In [ ]:
costs = costs[str(CLUSTERS)].rename(lambda x: float(x), axis=1, level=0)

In [ ]:
df = (
    costs.xs(LV_OPTS, level="opt", axis=1)
    .xs("2050", level="planning_horizon", axis=1)
    .groupby(level=2)
    .sum()
    .div(1e9)
)

In [ ]:
df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

In [ ]:
df.sum()

In [ ]:
df.sum() - df.sum().max()

In [ ]:
benefit_abs = df.sum() - df.sum().max()
benefit_abs

In [ ]:
latex["lvbenefitabs"] = -benefit_abs[2]

In [ ]:
benefit_rel = (1 - df.sum() / df.sum().max()) * 100
benefit_rel

In [ ]:
latex["lvbenefitabs"] = -benefit_rel[2]

In [ ]:
to_drop = df.index[df.max(axis=1).fillna(0.0) < 1.2]
to_drop = to_drop.difference(
    ["DAC", "hot water storage"]
)  # exclude dropping to align with onwind sensitivity...
df.drop(to_drop, inplace=True)

In [ ]:
order = preferred_order.intersection(df.index).append(
    df.index.difference(preferred_order)
)
df = df.loc[order]

In [ ]:
df.columns *= 100

In [ ]:
dec = load_decentral()["H2 grid"]
dec.name = "\nno power\ntransmission"
dec = pd.DataFrame(dec).T

In [ ]:
dec.sum().sum()

In [ ]:
fig, (ax0, ax1) = plt.subplots(
    1,
    2,
    figsize=(5, 3),
    sharey=True,
    gridspec_kw={"width_ratios": [1, 6], "wspace": 0.05},
)

to_plot = df.T.sort_index()

tech_colors = config["plotting"]["tech_colors"]
colors = [tech_colors[i] for i in df.index]

to_plot.plot.area(ax=ax1, stacked=True, linewidth=0, color=colors)

dec.plot.bar(ax=ax0, stacked=True, linewidth=0, legend=False, color=tech_colors)

handles, labels = ax1.get_legend_handles_labels()

handles.reverse()
labels.reverse()

ax1.set_xlim(100, 200)
ax1.set_xlabel("Power Grid Reinforcement Restriction\n[% relative to today's volume]")

ax0.set_ylim([0, 1000])
ax0.set_ylabel("System Cost\n[EUR billion per year]")
ax0.grid(axis="y")
ax1.grid(axis="y")

ax1.axvline(125, color="k", linestyle="--", linewidth=1)

ax1.text(127.5, 810, "TYNDP equivalent", size=11, color="k")

# legend on side
ax1.legend(handles, labels, ncol=2, frameon=False, bbox_to_anchor=(1.02, 1.03))

for i in ["top", "right", "left", "bottom"]:
    ax1.spines[i].set_visible(False)
    ax0.spines[i].set_visible(False)

# legend inside
# ax1.legend(handles, labels, ncol=3, frameon=False, bbox_to_anchor=(1.1, 1.48))

ax1.set_axisbelow(False)

ax0.tick_params(rotation=0, labelsize=10)
ax1.tick_params(rotation=0, labelsize=10)
ax1.set_xticks([100, 125, 150, 175, 200])

fig.savefig(OUTPUT + "lv-sensitivity.pdf", bbox_inches="tight")

## Onshore Wind Sensitivity

In [ ]:
costs = pd.read_csv(
    ONW_SCENARIOS + "/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
)

In [ ]:
costs = costs.xs(
    (str(CLUSTERS), "1.25", "2050"), level=["cluster", "lv", "planning_horizon"], axis=1
)

In [ ]:
costs.columns = [
    100 * float(re.search(r"onwind\+p([0-9.]*)", c).groups()[0]) for c in costs.columns
]

In [ ]:
df = costs.groupby(level=2).sum().div(1e9)

In [ ]:
df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

In [ ]:
df.sum()

In [ ]:
df.sum() - df.sum().max()

In [ ]:
(1 - df.sum() / df.sum().min()) * 100

In [ ]:
to_drop = df.index[df.max(axis=1).fillna(0.0) < 1.2]
print(to_drop)
df.drop(to_drop, inplace=True)

In [ ]:
order = preferred_order.intersection(df.index).append(
    df.index.difference(preferred_order)
)
df = df.loc[order]

In [ ]:
spec = load_main().T

spec = spec.xs(("1.0", 0.0, "H2 grid"), level=["lv", "onw", "h2"])

spec.index = ["no grid\nexpansion\nno onshore wind"]

In [ ]:
fig, (ax0, ax1) = plt.subplots(
    1,
    2,
    figsize=(5, 3),
    sharey=True,
    gridspec_kw={"width_ratios": [1, 6], "wspace": 0.05},
)

to_plot = df.T.sort_index()

tech_colors = config["plotting"]["tech_colors"]
colors = [tech_colors[i] for i in df.index]

to_plot.plot.area(ax=ax1, stacked=True, linewidth=0, color=colors)

spec.plot.bar(ax=ax0, stacked=True, linewidth=0, legend=False, color=tech_colors)

handles, labels = ax1.get_legend_handles_labels()

handles.reverse()
labels.reverse()

ax1.set_xlim(0, 100)
ax1.set_xlabel("Fraction of technical onshore\nwind potential available [%]")

ax0.set_ylim(0, 1000)
ax0.set_ylabel("System Cost\n[EUR billion per year]")
ax0.grid(axis="y")
ax1.grid(axis="y")

ax1.axvline(25, color="k", linestyle="--", linewidth=1)

ax1.text(27.5, 900, "compromise social potential", size=11, color="k")

# legend on side
ax1.legend(handles, labels, ncol=2, frameon=False, bbox_to_anchor=(1, 1.02))

for i in ["top", "right", "left", "bottom"]:
    ax1.spines[i].set_visible(False)
    ax0.spines[i].set_visible(False)

# legend inside
# ax.legend(handles, labels, ncol=3, frameon=False, bbox_to_anchor=(1.1, 1.48))

ax1.set_axisbelow(False)

ax0.tick_params(rotation=0, labelsize=10)
ax1.tick_params(rotation=0, labelsize=10)
ax1.set_xticks([0, 25, 50, 75, 100])

fig.savefig(OUTPUT + "onw-sensitivity.pdf", bbox_inches="tight")


## Inter-scenario diff charts

In [ ]:
def plot_diff(
    cost,
    main,
    df=None,
    legend=False,
    suffix=None,
    label="cost",
    unit="bn€/a",
    scaler=50,
    threshold=False,
    label_scaler=15,
    rename=True,
):

    union = cost.index.union(main.index)
    cost = cost.reindex(index=union, fill_value=0.0)
    main = main.reindex(index=union, fill_value=0.0)

    if df is None:
        df = cost - main

    if rename:
        column_dict = {
            "1.0": "w/o power grid expansion",
            "opt": "w power grid expansion",
            "H2 grid": "w hydrogen network",
            "no H2 grid": "w/o hydrogen network",
        }

        def fix_df(df):
            df.rename(columns=column_dict, inplace=True)
            df.columns = ["\n".join(col).strip() for col in df.columns.values]
            df.sort_index(axis=1, inplace=True)

        fix_df(df)
        fix_df(cost)
        fix_df(main)

    fig, ax = plt.subplots(1, 1, figsize=(6, 2.7))

    if threshold:
        to_drop = df.index[df.abs().max(axis=1).fillna(0.0) < threshold]
        df.drop(to_drop, inplace=True)

    df.round(1).T.plot.barh(
        ax=ax,
        stacked=True,
        color=tech_colors,
        xlabel=f"reduced"
        + r" $\leftarrow$ "
        + f"{label} [{unit}]"
        + r" $\rightarrow$ "
        + f"increased",
        legend=legend,
    )
    # ax.axvline(0, color='#454545', linewidth=1.5, zorder=-1)

    net = df.sum()
    rel = cost.sum() / main.sum() * 100 - 100
    netcolor = "darkslategrey"
    ax.scatter(net.values, net.index, s=10, c=netcolor, alpha=0.6, edgecolor="none")

    for x, y in net.items():
        ax.annotate(
            f"{y:.0f} | {rel[x]:.1f}%",
            (y, x),
            xytext=(-12, 13),
            textcoords="offset points",
            color=netcolor,
            fontsize=9,
            bbox=dict(fc="0.85", boxstyle="square,pad=0.1"),
        )

    plt.grid(axis="x")

    ylim = max(-df.where(df < 0).sum().min(), df.where(df > 0).sum().max())
    ylim = np.ceil(ylim / scaler) * scaler

    plt.xlim([-ylim, ylim])
    plt.ylim([-0.5, 3.8])

    for bars in ax.containers:
        labels = [
            f"{abs(v):.0f}" if abs(v) > ylim / label_scaler else ""
            for v in bars.datavalues
        ]
        ax.bar_label(
            bars, labels=labels, label_type="center", color="#444444", fontsize=9
        )

    handles = [
        plt.Line2D(
            [],
            [],
            color=mpl.colors.to_rgba(netcolor, 0.5),
            marker=".",
            markeredgecolor="none",
            linestyle="None",
            markersize=10,
        )
    ]
    legend = ax.legend(
        handles,
        ["net difference"],
        labelcolor=netcolor,
        fontsize=11,
        frameon=True,
        bbox_to_anchor=(-0.015, -0.055),
    )
    ax.add_artist(legend)

    plt.tight_layout()

    if legend:
        plt.legend(ncol=1, loc=(1.05, -0.25), labelspacing=0.2)

    plt.savefig(
        OUTPUT + f"diff-{label}{suffix}.pdf", bbox_inches=Bbox([[0, 0], [8.5, 2.8]])
    )

In [ ]:
for SCENARIO in [IMP_SCENARIOS, SHP_SCENARIOS, COST_SCENARIOS]:

    suffix = "-" + SCENARIO.split("-")[-1] if SCENARIO != MAIN_SCENARIOS else ""

    print(suffix)

    main = load_main(MAIN_SCENARIOS).xs(100.0, level="onw", axis=1)

    cost = load_main(SCENARIO).xs(100.0, level="onw", axis=1)

    plot_diff(cost, main, suffix=suffix, threshold=1)

    a = load_main_energy(SCENARIO).xs(100.0, level="onw", axis=1)
    b = load_main_energy(MAIN_SCENARIOS).xs(100.0, level="onw", axis=1)

    a_pos = a.where(a > 0)
    b_pos = b.where(b > 0)

    plot_diff(
        a_pos,
        b_pos,
        suffix=suffix,
        label="supply",
        unit=r"TWh & Mt$_{CO_2}$",
        scaler=200,
        threshold=0.1,
        label_scaler=14,
    )

    a_neg = -a.where(a < 0)
    b_neg = -b.where(b < 0)

    plot_diff(
        a_neg,
        b_neg,
        suffix=suffix,
        label="consumption",
        unit=r"TWh & Mt$_{CO_2}$",
        scaler=500,
        legend=True,
        threshold=0.1,
        label_scaler=8,
    )

    gw_a, twh_a = load_main_capacities(SCENARIO)
    gw_b, twh_b = load_main_capacities(MAIN_SCENARIOS)

    gw_a = gw_a.xs(100.0, level="onw", axis=1)
    gw_b = gw_b.xs(100.0, level="onw", axis=1)
    twh_a = twh_a.xs(100.0, level="onw", axis=1)
    twh_b = twh_b.xs(100.0, level="onw", axis=1)

    plot_diff(
        gw_a,
        gw_b,
        suffix=suffix,
        label="capacity",
        unit="GW",
        scaler=250,
        threshold=1,
        label_scaler=8,
    )

    plot_diff(
        twh_a, twh_b, suffix=suffix, label="storage", unit="TWh", scaler=40, threshold=1
    )

## Time diff charts

In [ ]:
def plot_time_diff(
    df,
    reference,
    legend=False,
    label="cost",
    unit="bn€/a",
    scaler=50,
    threshold=False,
    label_scaler=15,
    rename=True,
    ylim=None,
):

    ref = df[reference]

    df = (df.T - df[reference]).drop(reference).T

    df.columns = [f"{c}-hourly\n{8760/c:.0f} snapshots" for c in df.columns]

    fig, ax = plt.subplots(1, 1, figsize=(6, 3.4))

    if threshold:
        to_drop = df.index[df.abs().max(axis=1).fillna(0.0) < threshold]
        df.drop(to_drop, inplace=True)

    ax.axhline(1, zorder=-1, alpha=0.1, linewidth=35, color="darkslategray")
    df.round(1).T.plot.barh(
        ax=ax,
        stacked=True,
        color=tech_colors,
        xlabel=f"reduced"
        + r" $\leftarrow$ "
        + f"{label} [{unit}]"
        + r" $\rightarrow$ "
        + f"increased",
        legend=legend,
    )

    net = df.sum()
    rel = df.sum() / ref.sum() * 100

    netcolor = "darkslategrey"
    ax.scatter(net.values, net.index, s=10, c=netcolor, alpha=0.6, edgecolor="none")

    for x, y in net.items():
        ax.annotate(
            f"{y:.1f} | {rel[x]:.2f}%",
            (y, x),
            xytext=(-12, 13),
            textcoords="offset points",
            color=netcolor,
            fontsize=9,
            bbox=dict(fc="0.85", boxstyle="square,pad=0.1"),
        )

    plt.grid(axis="x")

    if ylim is None:
        ylim = max(-df.where(df < 0).sum().min(), df.where(df > 0).sum().max())
        ylim = np.ceil(ylim / scaler) * scaler

    plt.xlim([-ylim, ylim])
    plt.ylim([-0.5, 4.8])
    plt.ylabel("")

    for bars in ax.containers:
        labels = [
            f"{abs(v):.0f}" if abs(v) > ylim / label_scaler else ""
            for v in bars.datavalues
        ]
        ax.bar_label(
            bars, labels=labels, label_type="center", color="#444444", fontsize=9
        )

    handles = [
        plt.Line2D(
            [],
            [],
            color=mpl.colors.to_rgba(netcolor, 0.5),
            marker=".",
            markeredgecolor="none",
            linestyle="None",
            markersize=10,
        )
    ]
    legend = ax.legend(
        handles,
        ["net difference"],
        labelcolor=netcolor,
        fontsize=10,
        frameon=False,
        loc=(1.05, -0.1),
    )
    ax.add_artist(legend)

    plt.tight_layout()

    if legend:
        plt.legend(ncol=1, loc=(1.05, 0), labelspacing=0.2)

    plt.savefig(
        OUTPUT + f"diff-time-{grid.replace('.', 'p')}-{label}.pdf",
        bbox_inches=Bbox([[0, 0], [8.5, 3.4]]),
    )

In [ ]:
grid = "1.0"

In [ ]:
df = (
    load_main(TIME_SCENARIOS, clusters=90, with_resolution=True)
    .xs(grid, level="lv", axis=1)
    .droplevel(["onw", "h2"], axis=1)
)

In [ ]:
plot_time_diff(df, 1, rename=False, scaler=20, label_scaler=50, threshold=0.1, ylim=60)

In [ ]:
df = (
    load_main_energy(TIME_SCENARIOS, clusters=90, with_resolution=True)
    .xs(grid, level="lv", axis=1)
    .droplevel(["onw", "h2"], axis=1)
)

In [ ]:
plot_time_diff(
    df.where(df > 0),
    1,
    rename=False,
    scaler=50,
    label_scaler=25,
    threshold=0.1,
    label="supply",
    unit=r"TWh & Mt$_{CO_2}$",
    ylim=700,
)

In [ ]:
plot_time_diff(
    -df.where(df < 0),
    1,
    rename=False,
    scaler=50,
    label_scaler=25,
    threshold=0.1,
    label="consumption",
    unit=r"TWh & Mt$_{CO_2}$",
    ylim=120,
)

In [ ]:
gw, twh = load_main_capacities(TIME_SCENARIOS, clusters=90, with_resolution=True)

In [ ]:
gw = gw.xs(grid, level="lv", axis=1).droplevel(["onw", "h2"], axis=1)
twh = twh.xs(grid, level="lv", axis=1).droplevel(["onw", "h2"], axis=1)

In [ ]:
plot_time_diff(
    gw,
    1,
    scaler=50,
    label_scaler=15,
    threshold=0.1,
    label="capacity",
    unit=r"GW",
    ylim=600,
)

In [ ]:
plot_time_diff(
    twh,
    1,
    scaler=50,
    label_scaler=25,
    threshold=0.1,
    label="storage",
    unit=r"TWh",
    ylim=40,
)

## Spatial diff charts

In [ ]:
def plot_space_diff(
    df,
    reference,
    legend=False,
    label="cost",
    unit="bn€/a",
    scaler=50,
    threshold=False,
    label_scaler=15,
    rename=True,
    ylim=None,
):

    ref = df[reference]

    df = (df.T - df[reference]).drop(reference).T

    df.columns = [f"{c} regions" for c in df.columns]

    fig, ax = plt.subplots(1, 1, figsize=(6, 2.8))

    if threshold:
        to_drop = df.index[df.abs().max(axis=1).fillna(0.0) < threshold]
        df.drop(to_drop, inplace=True)

    # ax.axhline(1, zorder=-1, alpha=0.1, linewidth=35, color='darkslategray')
    df.round(1).T.plot.barh(
        ax=ax,
        stacked=True,
        color=tech_colors,
        xlabel=f"reduced"
        + r" $\leftarrow$ "
        + f"{label} [{unit}]"
        + r" $\rightarrow$ "
        + f"increased",
        legend=legend,
    )

    net = df.sum()
    rel = df.sum() / ref.sum() * 100

    netcolor = "darkslategrey"
    ax.scatter(net.values, net.index, s=10, c=netcolor, alpha=0.6, edgecolor="none")

    for x, y in net.items():
        ax.annotate(
            f"{y:.1f} | {rel[x]:.2f}%",
            (y, x),
            xytext=(-12, 13),
            textcoords="offset points",
            color=netcolor,
            fontsize=9,
            bbox=dict(fc="0.85", boxstyle="square,pad=0.1"),
        )

    plt.grid(axis="x")

    if ylim is None:
        ylim = max(-df.where(df < 0).sum().min(), df.where(df > 0).sum().max())
        ylim = np.ceil(ylim / scaler) * scaler

    plt.xlim([-ylim, ylim])
    plt.ylim([-0.5, 3.8])
    plt.ylabel("")

    for bars in ax.containers:
        labels = [
            f"{abs(v):.0f}" if abs(v) > ylim / label_scaler else ""
            for v in bars.datavalues
        ]
        ax.bar_label(
            bars, labels=labels, label_type="center", color="#444444", fontsize=9
        )

    handles = [
        plt.Line2D(
            [],
            [],
            color=mpl.colors.to_rgba(netcolor, 0.5),
            marker=".",
            markeredgecolor="none",
            linestyle="None",
            markersize=10,
        )
    ]
    legend = ax.legend(
        handles,
        ["net difference"],
        labelcolor=netcolor,
        fontsize=10,
        frameon=False,
        loc=(1.05, -0.25),
    )
    ax.add_artist(legend)

    plt.tight_layout()

    if legend:
        plt.legend(ncol=1, loc=(1.05, -0.15), labelspacing=0.2)

    plt.savefig(
        OUTPUT
        + f"diff-space-{grid.replace('.', 'p')}-{h2.replace(' ', '_')}-{label}.pdf",
        bbox_inches=Bbox([[0, 0], [8.5, 2.8]]),
    )

In [ ]:
grid = "1.0"
h2 = "no H2 grid"
df = (
    load_main(SPACE_SCENARIOS, with_space=True)
    .xs((grid, h2), level=["lv", "h2"], axis=1)
    .droplevel("onw", axis=1)
)
s = load_main(MAIN_SCENARIOS)[(grid, 100, h2)]
s.name = "181"
df = pd.concat([df, s], axis=1).fillna(0.0).loc[:, ::-1]

In [ ]:
plot_space_diff(
    df, "181", rename=False, scaler=20, label_scaler=30, threshold=0.1, ylim=60
)

In [ ]:
df = (
    load_main_energy(SPACE_SCENARIOS, with_space=True)
    .xs((grid, h2), level=["lv", "h2"], axis=1)
    .droplevel("onw", axis=1)
)
s = load_main_energy(MAIN_SCENARIOS)[(grid, 100, h2)]
s.name = "181"
df = pd.concat([df, s], axis=1).fillna(0.0).loc[:, ::-1]

In [ ]:
plot_space_diff(
    df.where(df > 0),
    "181",
    rename=False,
    scaler=50,
    label_scaler=25,
    threshold=0.1,
    label="supply",
    unit=r"TWh & Mt$_{CO_2}$",
    ylim=700,
)

In [ ]:
plot_space_diff(
    -df.where(df < 0),
    "181",
    rename=False,
    scaler=50,
    label_scaler=25,
    threshold=0.1,
    label="consumption",
    unit=r"TWh & Mt$_{CO_2}$",
    ylim=250,
)

In [ ]:
gw, twh = load_main_capacities(SPACE_SCENARIOS, with_space=True)
gw_, twh_ = load_main_capacities(MAIN_SCENARIOS)

gw = gw.xs((grid, h2), level=["lv", "h2"], axis=1).droplevel("onw", axis=1)
gw_ = gw_[(grid, 100, h2)]

twh = twh.xs((grid, h2), level=["lv", "h2"], axis=1).droplevel("onw", axis=1)
twh_ = twh_[(grid, 100, h2)]

gw_.name = "181"
twh_.name = "181"

gw = pd.concat([gw, gw_], axis=1).fillna(0.0).loc[:, ::-1]
twh = pd.concat([twh, twh_], axis=1).fillna(0.0).loc[:, ::-1]

In [ ]:
plot_space_diff(
    gw,
    "181",
    scaler=50,
    label_scaler=15,
    threshold=0.1,
    label="capacity",
    unit=r"GW",
    ylim=600,
)

In [ ]:
plot_space_diff(
    twh,
    "181",
    scaler=50,
    label_scaler=25,
    threshold=0.1,
    label="storage",
    unit=r"TWh",
    ylim=40,
)

## Intra-scenario diff

In [ ]:
def plot_internal_diff(
    df,
    reference,
    legend=False,
    label="cost",
    unit="bn€/a",
    scaler=50,
    threshold=False,
    label_scaler=15,
    rename=True,
):

    ref = df[reference]

    df = (df.T - df[reference]).drop(reference).T

    if rename:
        column_dict = {
            "1.0": "w/o power grid expansion",
            "opt": "",
            "H2 grid": "",
            "no H2 grid": "w/o hydrogen network",
        }

        df.rename(columns=column_dict, inplace=True)
        df.columns = ["\n".join(col).strip() for col in df.columns.values]
        df.sort_index(axis=1, inplace=True)

    fig, ax = plt.subplots(1, 1, figsize=(6.7, 2.4))

    if threshold:
        to_drop = df.index[df.abs().max(axis=1).fillna(0.0) < threshold]
        df.drop(to_drop, inplace=True)

    df.round(1).T.plot.barh(
        ax=ax,
        stacked=True,
        color=tech_colors,
        xlabel=f"reduced"
        + r" $\leftarrow$ "
        + f"{label} [{unit}]"
        + r" $\rightarrow$ "
        + f"increased",
        legend=legend,
    )

    net = df.sum()
    rel = df.sum() / ref.sum() * 100

    netcolor = "darkslategrey"
    ax.scatter(net.values, net.index, s=10, c=netcolor, alpha=0.6, edgecolor="none")

    for x, y in net.items():
        ax.annotate(
            f"{y:.1f} | {rel[x]:.1f}%",
            (y, x),
            xytext=(-12, 13),
            textcoords="offset points",
            color=netcolor,
            fontsize=9,
            bbox=dict(fc="0.85", boxstyle="square,pad=0.1"),
        )

    plt.grid(axis="x")

    ylim = max(-df.where(df < 0).sum().min(), df.where(df > 0).sum().max())
    ylim = np.ceil(ylim / scaler) * scaler

    plt.xlim([-ylim, ylim])
    plt.ylim([-0.5, 2.8])
    plt.ylabel("")

    for bars in ax.containers:
        labels = [
            f"{abs(v):.0f}" if abs(v) > ylim / label_scaler else ""
            for v in bars.datavalues
        ]
        ax.bar_label(
            bars, labels=labels, label_type="center", color="#444444", fontsize=9
        )

    handles = [
        plt.Line2D(
            [],
            [],
            color=mpl.colors.to_rgba(netcolor, 0.5),
            marker=".",
            markeredgecolor="none",
            linestyle="None",
            markersize=10,
        )
    ]
    legend = ax.legend(
        handles,
        ["net difference"],
        labelcolor=netcolor,
        fontsize=11,
        frameon=True,
        loc=(-0.33, -0.3),
    )
    ax.add_artist(legend)

    plt.tight_layout()

    if legend:
        plt.legend(ncol=3, loc=(-0.35, 1.01), labelspacing=0.2)

    plt.savefig(
        OUTPUT + f"diff-internal-{label}{suffix}.pdf",
        bbox_inches=Bbox([[0, 0], [7.3, 3.8]]),
    )

In [ ]:
reference = ("opt", "H2 grid")

for SCENARIO in [MAIN_SCENARIOS, IMP_SCENARIOS, SHP_SCENARIOS, COST_SCENARIOS]:

    suffix = "-" + SCENARIO.split("-")[-1] if SCENARIO != MAIN_SCENARIOS else ""

    df = load_main(SCENARIO).xs(100.0, level="onw", axis=1)

    plot_internal_diff(df, reference, scaler=40, label_scaler=25, threshold=0.5)

    df = load_main_energy(SCENARIO).xs(100.0, level="onw", axis=1)

    plot_internal_diff(
        df.where(df > 0),
        reference,
        scaler=200,
        label_scaler=15,
        threshold=0.1,
        label="supply",
        unit=r"TWh & Mt$_{CO_2}$",
    )

    plot_internal_diff(
        -df.where(df < 0),
        reference,
        scaler=200,
        label_scaler=10,
        threshold=0.1,
        label="consumption",
        unit=r"TWh & Mt$_{CO_2}$",
    )

    gw, twh = load_main_capacities(SCENARIO)

    df = gw.xs(100.0, level="onw", axis=1)

    plot_internal_diff(
        df,
        reference,
        threshold=1,
        scaler=500,
        label_scaler=10,
        label="capacity",
        unit="GW",
    )

    df = twh.xs(100.0, level="onw", axis=1)

    plot_internal_diff(
        df, reference, threshold=1, scaler=60, label="storage", unit="TWh"
    )

## H2 Network Scenarios

In [ ]:
SCENARIO = MAIN_SCENARIOS

df = load_main(SCENARIO)

suffix = "-" + SCENARIO.split("-")[-1] if SCENARIO != MAIN_SCENARIOS else ""

In [ ]:
df.sum().sort_values()

In [ ]:
dff = df.sum().unstack("h2")

dff

In [ ]:
rel_advantage = (((dff["no H2 grid"] / dff["H2 grid"] - 1) * 100).round(1)).unstack().T
rel_advantage

In [ ]:
abs_advantage = ((dff["no H2 grid"] - dff["H2 grid"]).round().astype(int)).unstack().T
abs_advantage

In [ ]:
(df.sum() - df.sum().max()).sort_values().round()

In [ ]:
((1 - df.sum() / df.sum().max()) * 100).sort_values().round(1)

In [ ]:
x = df.sum().xs(100.0, level="onw")
xx = x / x.min()
xx.unstack().round(3)

In [ ]:
dff.xs(100.0, level="onw")

In [ ]:
tech_colors = config["plotting"]["tech_colors"]
colors = [tech_colors[i] for i in df.index]

colors_list = [
    "hydroelectricity",
    "onshore wind",
    "offshore wind",
    "solar PV",
    # "solar thermal",
    "solar rooftop",
    "power-to-heat",
    "gas-to-power/heat",
    "power-to-hydrogen",
    "power-to-liquid",
    "battery storage",
    "hot water storage",
    "carbon capture",
    "H2 storage",
    "steam methane reforming",
    "steam methane reforming CC",
    "methanation",
    "electricity distribution grid",
    "fossil oil and gas",
    "biogas",
    "solid biomass",
    "direct air capture",
]

color_map = {i: tech_colors[i] for i in colors_list}
color_map["HVDC transmission"] = "darkseagreen"
color_map["HVAC transmission"] = "rosybrown"

color_map = pd.Series(color_map).sort_index(key=lambda x: x.map(str.lower)).to_dict()

fig, ax = plt.subplots(figsize=(10, 1.5))
handles = [mpatches.Patch(color=v, label=k) for k, v in color_map.items()]
ax.legend(handles=handles, ncol=6)
ax.axis("off")
plt.savefig(OUTPUT + "color_legend.pdf", bbox_inches="tight")

In [ ]:
xx = enumerate(df.columns.get_level_values("lv").unique()[::-1])
yy = enumerate(df.columns.get_level_values("onw").unique())

fig, axs = plt.subplots(2, 2, figsize=(5.5, 5.5), sharex=True, sharey=True)

kwargs = dict(stacked=True, color=tech_colors, ylim=(0, 1100), legend=False)

for x, y in product(xx, yy):

    ax = axs[x[0], y[0]]

    toprow_kwargs = (
        dict(title=f"onshore wind\n{int(y[1])}% potential\n") if x[0] == 0 else {}
    )

    ylabel_value = "optimal" if x[1] == "opt" else f"{100 * float(x[1]) - 100:.{0}f}%"

    to_plot = df.xs((x[1], y[1]), axis=1, level=["lv", "onw"]).T.sort_index(
        ascending=True
    )
    to_plot.plot.bar(
        ax=ax,
        **kwargs,
        ylabel=f"line expansion\n{ylabel_value}\n\nbn€/a",
        **toprow_kwargs,
    )

    ax.set_xlabel("", rotation=0)

    ax.tick_params(labelrotation=0)

    ax.grid(axis="y")
    ax.title.set_size(11)

    # ax.patch.set_visible(False)

    for i in ["top", "right", "left", "bottom"]:
        ax.spines[i].set_visible(False)

    label = f"+ {rel_advantage.loc[y[1],x[1]]}%\n+ {abs_advantage.loc[y[1],x[1]]}"
    ax.text(0.31, 825, label, size=8.5, color="k")


handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
fig.legend(handles, labels, bbox_to_anchor=(1.45, 0.9))

plt.tight_layout()

plt.savefig(f"{OUTPUT}sensitivity-h2{suffix}.pdf", bbox_inches="tight")

In [ ]:
df_new = df.xs(100, level="onw", axis=1)

dff = df_new.sum().unstack("h2")

dff / dff.min(axis=0)

In [ ]:
h2_rel_benefit = (dff.T / dff.min(axis=1) * 100 - 100).iloc[1].reset_index(drop=True)
h2_abs_benefit = (dff.T - dff.min(axis=1)).iloc[1].reset_index(drop=True)

ac_rel_benefit = (dff / dff.min(axis=0) * 100 - 100).iloc[0].reset_index(drop=True)
ac_abs_benefit = (dff - dff.min(axis=0)).iloc[0].reset_index(drop=True)

max_rel_benefit = dff.max().max() / dff.min().min() * 100 - 100

max_abs_benefit = int(dff.max().max() - dff.min().min())

In [ ]:
ac_grid_cost = df.xs(100.0, level="onw", axis=1).loc["transmission lines"].round(1)
h2_grid_cost = df.xs(100.0, level="onw", axis=1).loc["H2 pipeline"].round(1)

In [ ]:
heavy_grouping = "-grouped"
if heavy_grouping == "-grouped":

    def heavy_aggregation(i):
        if "solar" in i:
            return "solar"
        if "bio" in i:
            return "biomass"

        if i in ["power-to-hydrogen", "methanation", "power-to-liquid"]:
            return "power-to-gas/liquid"
        if "capture" in i or "CC" in i:
            return "carbon capture"
        if i in ["H2 pipeline", "electricity distribution grid", "transmission lines"]:
            return "transmission"
        if "storage" in i:
            return "storage"
        else:
            return i

    df_new = df_new.groupby(heavy_aggregation).sum()
    tech_colors["transmission"] = tech_colors["transmission lines"]
    tech_colors["storage"] = tech_colors["battery storage"]
    tech_colors["power-to-gas/liquid"] = tech_colors["power-to-gas"]
    df_new = df_new.sort_values(by=("opt", "H2 grid"), ascending=False)

In [ ]:
xx = enumerate(df_new.columns.get_level_values("lv").unique()[::-1])
yy = enumerate(df_new.columns.get_level_values("h2").unique())

fig, axs = plt.subplots(2, 2, figsize=(4, 6), sharey=True)

plt.subplots_adjust(hspace=0.5, wspace=1)

kwargs = dict(stacked=True, color=tech_colors, ylim=(0, 900), legend=False)

for x, y in product(xx, yy):

    ax = axs[x[0], y[0]]

    toprow_kwargs = (
        dict(
            title="with\nhydrogen grid\n"
            if y[1] == "H2 grid"
            else "without\nhydrogen grid\n"
        )
        if x[0] == 0
        else {}
    )

    ylabel = (
        "with power\ngrid expansion\n\nbn€/a"
        if x[1] == "opt"
        else "without power\ngrid expansion\n\nbn€/a"
    )

    to_plot = df_new.xs((x[1], y[1]), axis=1, level=["lv", "h2"]).T

    to_plot.plot.bar(
        ax=ax,
        ylabel=ylabel,
        **kwargs,
        **toprow_kwargs,
    )

    ax.set_xlabel("", rotation=0)
    ax.set_xticks([], [])

    ax.tick_params(labelrotation=0)

    ax.grid(axis="y")
    ax.title.set_size(11)

    ax.set_yticks(np.arange(0, 901, 100))

    print()

    ax.text(-0.3, 825, f"{to_plot.sum().sum():.0f} bn€/a", color="grey", fontsize=9.5)

    for i in ["top", "right", "left", "bottom"]:
        ax.spines[i].set_visible(False)

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
fig.legend(handles, labels, bbox_to_anchor=(1.4, 0.9))

fig.text(
    0.41,
    0.3,
    f"+ {h2_rel_benefit[0]:.1f}%\n+ {h2_abs_benefit[0]:.0f} bn€/a",
    fontsize=11,
)
fig.text(
    0.41,
    0.76,
    f"+ {h2_rel_benefit[1]:.1f}%\n+ {h2_abs_benefit[1]:.0f} bn€/a",
    fontsize=11,
)
fig.text(
    0.11,
    0.47,
    f"+ {ac_rel_benefit[0]:.1f}%\n+ {ac_abs_benefit[0]:.0f}\nbn€/a",
    fontsize=11,
)
fig.text(
    0.8,
    0.47,
    f"+ {ac_rel_benefit[1]:.1f}%\n+ {ac_abs_benefit[1]:.0f}\nbn€/a",
    fontsize=11,
)
fig.text(
    0.33,
    0.47,
    f"+ {max_rel_benefit:.1f}%\n+ {max_abs_benefit} bn€/a",
    fontsize=11,
    color="grey",
)


def add_arrow(ax0, ax1, pos0, pos1, **arrow_kwargs):
    ax0tr = ax0.transData  # Axis 0 -> Display
    ax1tr = ax1.transData  # Axis 1 -> Display
    figtr = fig.transFigure.inverted()  # Display -> Figure
    ptB = figtr.transform(ax0tr.transform(pos0))
    ptE = figtr.transform(ax1tr.transform(pos1))
    arrow = FancyArrowPatch(
        ptB,
        ptE,
        transform=fig.transFigure,  # Place arrow in figure coord system
        **arrow_kwargs,
    )
    fig.patches.append(arrow)


norm = mpl.colors.Normalize(vmin=0, vmax=10)
m = cm.ScalarMappable(norm=norm, cmap=cm.cividis)

arrow_style = dict(arrowstyle="simple", mutation_scale=22, ec=None)

add_arrow(
    axs[0, 0],
    axs[0, 1],
    (0.5, 500),
    (-0.5, 500),
    fc=m.to_rgba(h2_rel_benefit[1]),
    **arrow_style,
)

add_arrow(
    axs[1, 0],
    axs[1, 1],
    (0.5, 500),
    (-0.5, 500),
    fc=m.to_rgba(h2_rel_benefit[0]),
    **arrow_style,
)

add_arrow(
    axs[0, 0],
    axs[1, 0],
    (0, 0),
    (0, 900),
    fc=m.to_rgba(ac_rel_benefit[0]),
    **arrow_style,
)

add_arrow(
    axs[0, 1],
    axs[1, 1],
    (0, 0),
    (0, 900),
    fc=m.to_rgba(ac_rel_benefit[1]),
    **arrow_style,
)

add_arrow(
    axs[0, 0],
    axs[1, 1],
    (0.5, 0),
    (-0.5, 900),
    fc=m.to_rgba(max_rel_benefit),
    **arrow_style,
)

# plt.tight_layout()

plt.savefig(
    f"{OUTPUT}sensitivity-h2-new{suffix}{heavy_grouping}.pdf", bbox_inches="tight"
)

In [ ]:
dff.round(0).astype(int).loc["opt", "H2 grid"]

In [ ]:
latex["gridbenefitabs"] = max_abs_benefit
latex["gridbenefitrel"] = np.round(max_rel_benefit, 1)
latex["minacbenefitabs"] = ac_abs_benefit.round().astype(int).min()
latex["maxacbenefitabs"] = ac_abs_benefit.round().astype(int).max()
latex["minhybenefitabs"] = h2_abs_benefit.round().astype(int).min()
latex["maxhybenefitabs"] = h2_abs_benefit.round().astype(int).max()
latex["minacbenefitrel"] = ac_rel_benefit.round(1).min()
latex["maxacbenefitrel"] = ac_rel_benefit.round(1).max()
latex["minhybenefitrel"] = h2_rel_benefit.round(1).min()
latex["maxhybenefitrel"] = h2_rel_benefit.round(1).max()
latex["minsystemcost"] = dff.round(0).astype(int).min().min()
latex["maxsystemcost"] = dff.round(0).astype(int).max().max()

tsc = dff.round(0).astype(int)
latex["acvshycost"] = np.round(
    tsc.loc["1.0", "H2 grid"] / tsc.loc["opt", "no H2 grid"] * 100 - 100, 1
)

latex["minaccost"] = ac_grid_cost.min()
latex["maxaccost"] = ac_grid_cost.max()
latex["minhycost"] = h2_grid_cost.xs("H2 grid", level="h2").min()
latex["maxhycost"] = h2_grid_cost.xs("H2 grid", level="h2").max()
latex["benefithyofac"] = np.round(
    latex["maxhybenefitabs"] / latex["maxacbenefitabs"] * 100, 1
)
latex["additivebenefitabs"] = latex["maxhybenefitabs"] + latex["maxacbenefitabs"]
latex["additivebenefitrel"] = np.round(
    (latex["maxhybenefitabs"] + latex["maxacbenefitabs"])
    / latex["gridbenefitabs"]
    * 100
    - 100,
    1,
)

nohydro = (
    df[~df.index.str.contains("hydroelectricity")]
    .sum()
    .xs(100.0, level="onw")
    .astype(int)
)
latex["maxsystemcostnohydro"] = nohydro.max()
latex["minsystemcostnohydro"] = nohydro.min()

gw, twh = load_main_capacities(SCENARIO)

gw = gw.xs(100, level="onw", axis=1).round().astype(int)
twh = twh.xs(100, level="onw", axis=1).round().astype(int)

max_gw = gw.max(axis=1)

min_gw = gw.min(axis=1)

latex["minoffwind"] = min_gw["offshore wind"]
latex["maxoffwind"] = max_gw["offshore wind"]
latex["minonwind"] = min_gw["onshore wind"]
latex["maxonwind"] = max_gw["onshore wind"]
latex["minsolar"] = min_gw["solar PV"] + min_gw["solar rooftop"]
latex["maxsolar"] = max_gw["solar PV"] + max_gw["solar rooftop"]

rooftop_share = (
    gw.loc["solar rooftop"] / (gw.loc["solar PV"] + gw.loc["solar rooftop"]) * 100
)

latex["meanrooftopshare"] = int(rooftop_share.mean())
latex["meanutilityshare"] = 100 - latex["meanrooftopshare"]

offshore_share = (
    gw.loc["offshore wind"] / (gw.loc["offshore wind"] + gw.loc["onshore wind"]) * 100
)

latex["minoffshoreshare"] = int(offshore_share.min())
latex["maxoffshoreshare"] = int(offshore_share.max())

latex["minelectrolysis"] = gw.loc["power-to-hydrogen"].min()
latex["maxelectrolysis"] = gw.loc["power-to-hydrogen"].max()

cfs = load_main_cfs(SCENARIO).xs(100, level="onw", axis=1).mul(100).astype(int)

latex["mincfFT"] = cfs.loc["Fischer-Tropsch"].min()
latex["maxcfFT"] = cfs.loc["Fischer-Tropsch"].max()
latex["mincfelectrolysis"] = cfs.loc["H2 Electrolysis"].min()
latex["maxcfelectrolysis"] = cfs.loc["H2 Electrolysis"].max()

sel = ("opt", "H2 grid")

latex["utilisationAC"] = int(cfs.loc["AC", sel])

latex["utilisationHy"] = int(
    cfs.loc[["H2 pipeline retrofitted", "H2 pipeline"], sel].mean()
)

latex["hydrogenstorageacyhyy"] = twh.loc["H2 storage"][("opt", "H2 grid")]
latex["hydrogenstorageacyhyn"] = twh.loc["H2 storage"][("opt", "no H2 grid")]
latex["hydrogenstorageacnhyy"] = twh.loc["H2 storage"][("1.0", "H2 grid")]
latex["hydrogenstorageacnhyn"] = twh.loc["H2 storage"][("1.0", "no H2 grid")]

latex["thermalstoragemin"] = twh.loc["hot water storage"].min()
latex["thermalstoragemax"] = twh.loc["hot water storage"].max()

sankey = pd.read_csv(
    "../results/graphics-20221227/20221227-main/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/sankey.csv",
    index_col=0,
)
sankey.value = sankey.value.astype(int)

latex["hydrogenproduction"] = sankey.query("target == 'H2'").value.sum()

ptl = ["Fischer-Tropsch", "methanolisation"]
latex["ptlhydrogenusage"] = sankey.query("source == 'H2' and label in @ptl").value.sum()

latex["ptlwasteheat"] = sankey.query(
    "source == 'H2' and label in @ptl and target == 'heat'"
).value.sum()

latex["hydrogenindustrydemand"] = sankey.query(
    "source == 'H2' and target == 'H2 for industry demand'"
).value.sum()
latex["hydrogentransportdemand"] = sankey.query(
    "source == 'H2' and target == 'land transport fuel cell demand'"
).value.sum()

latex["hydrogenlosses"] = sankey.query(
    "source == 'H2' and target == 'losses'"
).value.sum()

sankey = pd.read_csv(
    "../results/graphics-20221227/20221227-main/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/sankey.csv",
    index_col=0,
)
sankey.value = sankey.value.astype(int)

latex["hydrogenfuelcell"] = sankey.query(
    "source == 'H2' and label == 'H2 Fuel Cell'"
).value.sum()

sankey = pd.read_csv(
    "../results/graphics-20221227/20221227-main/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/sankey.csv",
    index_col=0,
)
sankey.value = sankey.value.astype(int)

latex["hydrogenmethanation"] = sankey.query(
    "source == 'H2' and label == 'Sabatier'"
).value.sum()

latex["fossilgas"] = sankey.query("source == 'fossil gas'").value.sum()

latex["biogas"] = sankey.query("source == 'biogas'").value.sum()

smr = ["SMR CC", "SMR"]
latex["bluehydrogen"] = sankey.query("target == 'H2' and label in @smr").value.sum()

carbon = pd.read_csv(
    "../results/graphics-20221227/20221227-main/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/sankey-carbon.csv",
    index_col=0,
)
carbon.value = carbon.value.astype(int)

latex["mindac"] = carbon.query("label == 'DAC'").value.sum()

carbon = pd.read_csv(
    "../results/graphics-20221227/20221227-main/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/sankey-carbon.csv",
    index_col=0,
)
carbon.value = carbon.value.astype(int)

latex["maxdac"] = carbon.query("label == 'DAC'").value.sum()

latex["maxvres"] = (
    load_main_energy()
    .xs(100, level="onw", axis=1)
    .loc[["onshore wind", "offshore wind", "solar PV", "solar rooftop"]]
    .sum()
    .max()
)

latex["acoftotalbenefit"] = int(
    latex["maxacbenefitabs"] / latex["gridbenefitabs"] * 100
)
latex["hyoftotalbenefit"] = int(
    latex["maxhybenefitabs"] / latex["gridbenefitabs"] * 100
)

## H2 vs Electricity Grid

In [ ]:
levels = ["lv", "onw", "h2"]

In [ ]:
scenA = df.xs(("1.0", 100.0, "H2 grid"), level=levels, axis=1)
scenB = df.xs(("opt", 100.0, "no H2 grid"), level=levels, axis=1)
scenC = df.xs(("1.0", 100.0, "no H2 grid"), level=levels, axis=1)
scenD = df.xs(("opt", 100.0, "H2 grid"), level=levels, axis=1)

In [ ]:
scenA.columns = ["grid expansion\nelectricity: no\nhydrogen: yes"]
scenB.columns = ["grid expansion\nelectricity: yes\nhydrogen: no"]
scenC.columns = ["grid expansion\nelectricity: no\nhydrogen: no"]
scenD.columns = ["grid expansion\nelectricity: yes\nhydrogen: yes"]

In [ ]:
to_plot = pd.concat([scenD, scenB, scenA, scenC], axis=1).T

In [ ]:
tsc = to_plot.sum(axis=1)

In [ ]:
base = tsc["grid expansion\nelectricity: yes\nhydrogen: no"]

In [ ]:
diff_rel = (100 * tsc / base - 100).round(1)[
    "grid expansion\nelectricity: no\nhydrogen: yes"
]

In [ ]:
diff_abs = (tsc - base).round(1)["grid expansion\nelectricity: no\nhydrogen: yes"]

In [ ]:
diff_abs

In [ ]:
tsc.values

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))

to_plot.plot.bar(
    ax=ax,
    stacked=True,
    color=colors,
    ylim=(0, 900),
    ylabel="total system cost [bn€/a]",
)

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
plt.legend(handles, labels, bbox_to_anchor=(1, 1.02))

label = f"+ {diff_rel}%"
ax.text(1.28, 820, label, size=11, color="#444444")

label = f"+ {diff_abs} bn€/a"
ax.text(1.28, 860, label, size=11, color="k")

for i, v in enumerate(tsc.astype(int).values):
    ax.text(i - 0.07, v + 12, v, size=10, color="#444444")

ax.grid(axis="y")
plt.xticks(rotation=0, fontsize=11)
plt.yticks(np.arange(0, 901, 100), fontsize=11)

for i in ["top", "right", "left", "bottom"]:
    ax.spines[i].set_visible(False)

ax.axvline(0.5, color="k", linewidth=1.25, linestyle="--")
ax.axvline(2.5, color="k", linewidth=1.25, linestyle="--")

plt.tight_layout()

plt.savefig(OUTPUT + "h2-vs-elec-grid.pdf", bbox_inches="tight")

## Growing Exclusion

In [ ]:
kwargs = dict(level=["lv", "onw", "h2"], axis=1)

In [ ]:
costs = pd.read_csv(
    DEC_SCENARIOS + "/csvs/costs.csv", header=[0, 1, 2, 3], index_col=[0, 1, 2]
)

In [ ]:
costs = costs.xs((str(CLUSTERS), "2050"), level=["cluster", "planning_horizon"], axis=1)

In [ ]:
scenA = df.xs(("opt", 100.0, "H2 grid"), **kwargs)
scenB = df.xs(("1.0", 100.0, "H2 grid"), **kwargs)
scenC = df.xs(("1.0", 0.0, "H2 grid"), **kwargs)
scenD = df.xs(("1.0", 0.0, "no H2 grid"), **kwargs)

In [ ]:
scenA.columns = ["least-cost"]
scenB.columns = ["no grid expansion"]
scenC.columns = ["no grid expansion\nno onshore wind"]
scenD.columns = ["no grid expansion\nno onshore wind\nno hydrogen grid"]

In [ ]:
to_plot = pd.concat([scenA, scenB, scenC, scenD], axis=1).T

In [ ]:
to_plot.loc["today", "today"] = 700

order = to_plot.index[:-1].insert(0, to_plot.index[-1])
to_plot = to_plot.loc[order]

In [ ]:
to_plot.sum(axis=1) / to_plot.sum(axis=1)["least-cost"]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

to_plot.plot.bar(
    ax=ax,
    stacked=True,
    color=colors,
    ylim=(0, 1000),
    ylabel="total system cost [bn€/a]",
)

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
plt.legend(handles, labels, bbox_to_anchor=(1, 1.05))

ax.grid(axis="y")
plt.xticks(rotation=0, fontsize=11)
plt.yticks(np.arange(0, 1001, 100), fontsize=11)

for i in ["top", "right", "left", "bottom"]:
    ax.spines[i].set_visible(False)

plt.tight_layout()

plt.savefig(OUTPUT + "growing-exclusion.pdf", bbox_inches="tight")

## Decentral Scenarios

In [ ]:
df = load_decentral()

In [ ]:
to_plot = df.T

In [ ]:
tsc = to_plot.sum(axis=1)

In [ ]:
diff_rel = (100 * tsc / tsc.min() - 100).round(1)[1]

In [ ]:
diff_abs = (tsc - tsc.min()).round(1)[1]

In [ ]:
df.sum()

In [ ]:
tech_colors = config["plotting"]["tech_colors"]
colors = [tech_colors[i] for i in to_plot.columns]

In [ ]:
fig, ax = plt.subplots(figsize=(6.5, 5))

to_plot.plot.bar(
    ax=ax,
    stacked=True,
    color=colors,
    ylim=(0, 1000),
    ylabel="total system cost [bn€/a]",
)

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
plt.legend(handles, labels, bbox_to_anchor=(1, 1.02))

label = f"+ {diff_rel}%"
ax.text(0.2, 920, label, size=11, color="#444444")

label = f"+ {diff_abs} bn€/a"
ax.text(0.2, 960, label, size=11, color="k")

ax.grid(axis="y")
plt.xticks(rotation=0, fontsize=11)
plt.yticks(np.arange(0, 1001, 100), fontsize=11)

for i in ["top", "right", "left", "bottom"]:
    ax.spines[i].set_visible(False)

plt.tight_layout()

plt.savefig(OUTPUT + "decentral.pdf", bbox_inches="tight")

## TWkm & EWhkm

In [ ]:
def plot_network_stats(data, kind):

    fig, ax = plt.subplots(
        1, 2, sharey=True, figsize=(4, 3.5), gridspec_kw={"width_ratios": [1.6, 1]}
    )

    kwargs = dict(stacked=True, edgecolor="k", width=0.3)

    nindex = ["cost-optimal\nexpansion", "no power grid\nexpansion"]

    df = data.xs("yes", level="h2").filter(like="Electricity")[::-1]
    df.index = nindex
    color = ["#abd982", "#d2ebbc"] if kind == "ewhkm" else ["#6c9459", "#a6be9b"]
    df.plot.bar(ax=ax[0], color=color, position=1, **kwargs)

    dfh = data.xs("yes", level="h2").filter(like="Hydrogen")[::-1]
    dfh.index = nindex
    dfh.plot.bar(ax=ax[0], color=["#904d84", "#f081dc"], position=0, **kwargs)

    df2 = data.xs("no", level="h2")[::-1]
    df2.index = nindex[:2]
    df2.plot.bar(
        ax=ax[1],
        color=color,
        legend=False,
        edgecolor="k",
        width=0.5,
        stacked=True,
    )

    if kind == "twkm":
        ax[0].set_ylabel("TWkm")
        ax[0].set_ylim([0, 900])
        threshold = 50

    if kind == "ewhkm":
        ax[0].set_ylabel("EWhkm")
        ax[0].set_ylim([0, 4])
        threshold = 0.3

    ax[0].set_xlim([-0.5, 1.5])
    ax[0].set_xlabel("")
    ax[0].set_title(
        "H$_2$ network",
        color="#444444",
    )
    ax[1].set_title(
        "no H$_2$ network",
        color="#444444",
    )
    ax[1].set_xlabel("")

    ax[0].title.set_size(11)
    ax[1].title.set_size(11)

    ax[0].grid(axis="y")
    ax[1].grid(axis="y")

    for i in ["top", "right", "left", "bottom"]:
        ax[0].spines[i].set_visible(False)
        ax[1].spines[i].set_visible(False)

    for a in ax:
        for bars in a.containers:
            if kind == "ewhkm":
                labels = [f"{v:.2f}" if v > threshold else "" for v in bars.datavalues]
            else:
                labels = [f"{v:.0f}" if v > threshold else "" for v in bars.datavalues]
            a.bar_label(
                bars, labels=labels, label_type="center", color="#444444", fontsize=9
            )

    plt.tight_layout()

    ax[0].legend(bbox_to_anchor=(1.45, 1.6), ncol=1, labelcolor="#444444")

    suffix = COMMON_PATH.split("-")[-1]
    plt.savefig(f"{OUTPUT}/{kind}-{suffix}.pdf", bbox_inches="tight")

In [ ]:
COMMON_PATH = "../results/graphics-20221227/20221227-shipping"

twkm = (
    pd.concat(
        {
            ("opt", "yes"): pd.read_csv(
                f"{COMMON_PATH}/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/twkm.csv",
                index_col=[0, 1],
            ),
            ("opt", "no"): pd.read_csv(
                f"{COMMON_PATH}/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/twkm.csv",
                index_col=[0, 1],
            ),
            ("1.0", "yes"): pd.read_csv(
                f"{COMMON_PATH}/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/twkm.csv",
                index_col=[0, 1],
            ),
            ("1.0", "no"): pd.read_csv(
                f"{COMMON_PATH}/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/twkm.csv",
                index_col=[0, 1],
            ),
        },
        names=["lv", "h2"],
    )
    .squeeze()
    .unstack([2, 3])
)

In [ ]:
data = pd.DataFrame()

data["Electricity network existing"] = (
    twkm[("DC", "existing")] + twkm[("AC", "existing")]
)
data["Electricity network new"] = twkm[("DC", "added")] + twkm[("AC", "added")]
data["Hydrogen network retrofitted"] = twkm[("H2 pipeline retrofitted", "optimal")]
data["Hydrogen network new"] = twkm[("H2 pipeline", "optimal")]

In [ ]:
retro_share = (
    data["Hydrogen network retrofitted"]
    / data.filter(like="Hydrogen").sum(axis=1)
    * 100
).round(1)
latex["maxretroshare"] = retro_share.max()
latex["minretroshare"] = retro_share.min()

latex["maxtwkmelectricity"] = data.filter(like="Electricity").sum(axis=1).max()
latex["mintwkmelectricity"] = data.filter(like="Electricity").sum(axis=1).min()
latex["mintwkmhydrogen"] = data.filter(like="Hydrogen").sum(axis=1).min()
latex["maxtwkmhydrogen"] = data.filter(like="Hydrogen").sum(axis=1).max()

latex["twkmhigher"] = np.round(
    data["Electricity network new"].max()
    / data["Electricity network new"].replace(0, np.NaN).min()
    * 100
    - 100,
    1,
)

In [ ]:
plot_network_stats(data, "twkm")

In [ ]:
ewhkm = (
    pd.concat(
        {
            ("opt", "yes"): pd.read_csv(
                f"{COMMON_PATH}/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/ewhkm.csv",
                index_col=0,
            ),
            ("opt", "no"): pd.read_csv(
                f"{COMMON_PATH}/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/ewhkm.csv",
                index_col=0,
            ),
            ("1.0", "yes"): pd.read_csv(
                f"{COMMON_PATH}/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/ewhkm.csv",
                index_col=0,
            ),
            ("1.0", "no"): pd.read_csv(
                f"{COMMON_PATH}/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/ewhkm.csv",
                index_col=0,
            ),
        },
        names=["lv", "h2"],
    )
    .squeeze()
    .unstack(2)
)

In [ ]:
data = pd.DataFrame()

data["Electricity network (HVAC)"] = ewhkm["AC"]
data["Electricity network (HVDC)"] = ewhkm["DC"]
data["Hydrogen network retrofitted"] = ewhkm["H2 pipeline retrofitted"]
data["Hydrogen network new"] = ewhkm["H2 pipeline"]

In [ ]:
latex["ewhkmelectricity"] = data.loc[("1.0", "yes")].filter(like="Electricity").sum()
latex["ewhkmhydrogen"] = data.loc[("1.0", "yes")].filter(like="Hydrogen").sum()

In [ ]:
latex["ewhkmdiff"] = np.round(
    100 - data.loc[("1.0", "yes")].sum() / data.loc[("opt", "yes")].sum() * 100, 1
)

In [ ]:
plot_network_stats(data, "ewhkm")

## Capacities

In [ ]:
gw, twh = load_main_capacities(scenarios=MAIN_SCENARIOS, rename=False)
suffix = "-" + SCENARIO.split("-")[-1] if SCENARIO != MAIN_SCENARIOS else ""
df = gw.xs(100.0, axis=1, level="onw")

In [ ]:
index_dict = {
    "offwind-ac": "offshore wind (AC)",
    "offwind-dc": "offshore wind (DC)",
    "onwind": "onshore wind",
    "SMR": "steam methane reforming",
    "SMR CC": "steam methane reforming CC",
}

column_dict = {
    "1.0": "w/o power",
    "opt": "w power",
    "H2 grid": "w H$_2$",
    "no H2 grid": "w/o H$_2$",
}

df.rename(index=index_dict, columns=column_dict, inplace=True)

df.columns = ["\n".join(col).strip() for col in df.columns.values]

In [ ]:
df.sort_index(axis=1, inplace=True)

In [ ]:
tech_colors = config["plotting"]["tech_colors"]

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(10, 4))

groups = [
    ["solar rooftop", "solar"],
    ["onshore wind"],
    ["offshore wind (AC)", "offshore wind (DC)"],
    ["H2 Electrolysis", "steam methane reforming", "steam methane reforming CC"],
]

ylims = [
    [0, 4000],
    [0, 2000],
    [0, 300],
    [0, 1500],
]

for ax, group, ylim in zip(axs, groups, ylims):
    df.loc[group].T.plot.bar(ax=ax, stacked=True, color=tech_colors)
    # ax.set_xlabel('grid expansion')
    # ax.legend(bbox_to_anchor=(1.02, 1.4), labelcolor="#444444")
    ax.legend(loc=(-0.25, 1.05), labelcolor="#444444", labelspacing=0.25)
    ax.set_ylabel("capacity [GW]")
    ax.set_ylim(ylim)

# fig.supxlabel('grid expansion')
plt.tight_layout()
plt.savefig(OUTPUT + f"capacities{suffix}.pdf", bbox_inches="tight")

## Export Latex Variables

In [ ]:
with open("../paper/variables.tex", "w") as f:
    for k, v in latex.items():
        f.write("\\newcommand{\\" + k + "}{" + str(v) + "}\n")